In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
df = pd.read_csv("customer_feedback_clean.csv")

if 'label' not in df.columns:
    def simple_label(x):
        x = x.lower()
        if any(w in x for w in ['excellent','great','happy','satisfied','loved']):
            return "Positive"
        if any(w in x for w in ['delay','disappoint','refund','damaged','poor','wrong']):
            return "Negative"
        return "Neutral"
    df['label'] = df['feedback'].apply(simple_label)
    
label2id = {"Negative":0,"Neutral":1,"Positive":2}
id2label = {v:k for k,v in label2id.items()}
df['label_id'] = df['label'].map(label2id)

train_df, test_df = train_test_split(df[['processed','label_id']], test_size=0.2, random_state=42, stratify=df['label_id'])

train_ds = Dataset.from_pandas(train_df.rename(columns={'processed':'text'}))
test_ds  = Dataset.from_pandas(test_df.rename(columns={'processed':'text'}))

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)
train_ds.set_format(type='torch', columns=['input_ids','attention_mask','label_id'])
test_ds.set_format(type='torch', columns=['input_ids','attention_mask','label_id'])

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

args = TrainingArguments(
    "distilbert-finetuned-sentiment",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_total_limit=1,
    logging_steps=10,
    push_to_hub=False
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print(metrics)
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")
